In [1]:
#Importing packages we need
from numpy import genfromtxt #This is used to loading a csv-file as a numpy array
import numpy as np
import scipy.optimize as opt #used to numerically optimize
import matplotlib.pyplot as plt #pyplot is used to plot the data
import pandas as pd
from matplotlib import cm
import numdifftools as nd #Using this package to numerically approximate the hessian matrix
import time #Used to measure time to run code
from math import log

In [2]:
#This function simulates a ARCH(1)-process with normal errors 
#sigma and alpha are the relevant parameters in the modelling of the volatility.
def sim_ARCH1(omega, alpha, T):
    
    #Generating T random draws from a std. normal dist.
    z=np.random.normal(loc=0.0, scale=1.0, size=T) #loc determined mean, scale is the std deviaion and size is the number of simulations
    
    #Define the series x_t and sigma^2_t (sigma2) as empty variables we assign values to later. 
    x=np.empty(T)
    sigma2=np.empty(T)
    
    #Initiating x_0=0 and sigma2_0=1
    sigma2[0]=omega/(1-alpha) #initiating sigma2 as its long run average  
    x[0]=0
    
    #Simulating the ARCH(1) through a loop
    for t in range(1,T):
        sigma2[t]=omega+alpha*x[t-1]**2
        x[t]=np.sqrt(sigma2[t])*z[t]
        
    return x #output is a series with returns (x)

In [22]:
def ARCH1_filter(omega, alpha,x_values): #function to find sigma^2 from observations
    T=len(x_values)
    
    #Define the series sigma^2_t (sigma2) as empty variables we assign values to later. 
    sigma2=np.empty(T)
    
    sigma2[0]=omega
    
    for t in range(1,T):
        sigma2[t]=omega + alpha*x_values[t-1]**2
    
    return sigma2

def ARCH1_likelihood(omega,alpha,x_values):
    sigma2=ARCH1_filter(omega, alpha, x_values)
    
    LogL=np.sum(-0.5*np.log(sigma2)-0.5*x_values**2/sigma2) #note we have removed constant term
    
    return LogL

def optim(theta): #defining an optimization function that can change on parameters theta=(omega, alpha)
    ome,alp=theta
    return -ARCH1_likelihood(ome, alp,x_values) #note we are minizing the optim function so we take the minus ARCH1_likelihood

def ARCH1_likelihood_null(omega,alpha_null,x_values): #actually just a copy of the other likelihood function but hopefully highlights that we are optimizing with alpha fixed (under the null) and optimizing for both
    sigma2=ARCH1_filter(omega, alpha_null, x_values)
    
    LogL=np.sum(-0.5*np.log(sigma2)-0.5*x_values**2/sigma2) #note we have removed constant term
    
    return LogL
def optim_null(omega): #defining an optimization function that can now only change omega (i.e alpha_null is fixed)
    ome=omega
    return -ARCH1_likelihood_null(ome, alpha_null,x_values) #note we are minizing the optim function so we take the minus ARCH1_likelihood

In [23]:
start_time = time.time() #starting the timer
T=1000 #Sample length
alpha0=0 #True value of ARCH parameter
omega0=1 #True value of intercept in conditional variance equation
theta0=np.array([omega0,alpha0]) #initial guesses for omega, alpha (makes computation faster to guess right initially)
alpha_null=alpha0 #defining alpha null as the true parameter (could potentially also test wheter alpha_0=0 by setting alpha_null=0)
theta0_null=np.array([omega0]) #initial guess for optimization of omega only (when alpha_null is fixed)

M=1000 #number of monte carlo replications
sim=np.empty([M,5]) #variable to store realisations
z=np.random.normal(loc=0.0, scale=1.0, size=M) #loc determined mean, scale is the std deviaion and size is the number of simulations

for i in range(M):
    x_values=sim_ARCH1(omega0, alpha0, T)
    res=opt.minimize(optim, theta0,method='SLSQP', bounds=((0.0001,None),(0,None))) #Compute ML estimate (theta_hat) from simulated x above
    theta_hat=res.x #ML estimates (theta_hat=omega_hat,alpha_hat)
    res_null=opt.minimize(optim_null, theta0_null,method='SLSQP', bounds=[(0.0001,None)]) #ML estimate of omega_hat (with alpha_null fixed)
    omegahat_null=res_null.x[0] #Omega_hat estimate (under the null)
    LR=2*(ARCH1_likelihood(theta_hat[0],theta_hat[1],x_values)-ARCH1_likelihood_null(omegahat_null,alpha_null,x_values)) #computing LR ratio
    sim[i]=[theta_hat[1],LR,z[i],z[i]**2,max(z[i],0)**2] #note we only take alpha hat and not omega estimate
print('Elapsed time (all computations)+'"--- %s seconds ---" % (time.time() - start_time)) #ending timer and printing timestamp

Elapsed time (all computations)+--- 61.32831001281738 seconds ---


In [24]:
print("95-percentiles of (alphahat, LR, N(0,1), Chi2, HalfChi2) = "+ str(np.quantile(sim,0.95, axis=0)))

95-percentiles of (alphahat, LR, N(0,1), Chi2, HalfChi2) = [0.05175273 2.34024344 1.62214377 4.17366133 2.631387  ]
